In [ ]:
from enum import Enum

class BinaryOp(Enum):
    ADD = "+"
    SUB = "-"
    MUL = "*"
    DIV = "/"

class Expression(object):
    pass

class BinaryExpression(Expression):
    def __init__(self, op: BinaryOp, lhs: Expression, rhs: Expression) -> None:
        super().__init__()
        self.op = op
        self.lhs = lhs
        self.rhs = rhs

    def evaluate(self):
        pass